In [ ]:
!pip install sktime

In [ ]:
from sktime.datasets import (
	load_airline,
	load_unit_test,
    load_tecator,
)
from sktime.utils.plotting import plot_series
from sktime.utils._testing.hierarchical import _make_hierarchical
from sklearn.model_selection import train_test_split

import os
os.environ["JAX_PLATFORM_NAME"] = "cpu"
os.environ["JAX_PLATFORMS"] = "cpu"

import warnings
warnings.filterwarnings("ignore")

# Overview

List of all estimators, I have authored or co-authored

In [ ]:
from sktime.registry import all_estimators

all_estimators(
    filter_tags={"authors": "geetu040"},
    return_tags=['capability:global_forecasting', 'ignores-exogeneous-X'],
	return_names=True, as_dataframe=True, suppress_import_stdout=True
)

# AutoRegressiveWrapper

In [ ]:
y = load_airline()

In [ ]:
from sktime.forecasting.pytorchforecasting import PytorchForecastingNBeats

forecaster = PytorchForecastingNBeats(trainer_params={
    "max_epochs": 20,
})

In [ ]:
forecaster.get_tag("requires-fh-in-fit")

In [ ]:
forecaster.fit(y, fh=list(range(1, 40)))
preds = forecaster.predict()

In [ ]:
plot_series(y, preds)

In [ ]:
# forecaster.predict(fh=[1, 2, 3])
# forecaster.predict(fh=[2, 5])
# forecaster.predict(fh=list(range(1, 50)))

In [ ]:
from sktime.forecasting.compose import AutoRegressiveWrapper

forecaster = PytorchForecastingNBeats(trainer_params={
    "max_epochs": 20,
})
wrapper = AutoRegressiveWrapper(forecaster, horizon_length=5)

In [ ]:
wrapper.get_tag("requires-fh-in-fit")

In [ ]:
wrapper.fit(y)
preds = wrapper.predict(fh=list(range(1, 40)))

In [ ]:
plot_series(y, preds)

In [ ]:
forecaster.get_class_tag("capability:global_forecasting")

In [ ]:
wrapper.get_class_tag("capability:global_forecasting")

In [ ]:
wrapper.get_class_tag("ignores-exogeneous-X")

# LTSFTransformerForecaster

In [ ]:
from sktime.forecasting.ltsf import LTSFTransformerForecaster

y = load_airline()

y = (y - y.mean()) / y.std()

model = LTSFTransformerForecaster(
	seq_len=30,
	context_len=15,
	pred_len=15,
    num_epochs=50,
    batch_size=8,
    in_channels=1,
    individual=False,
    criterion=None,
    criterion_kwargs=None,
    optimizer=None,
    optimizer_kwargs=None,
    lr=0.002,
    position_encoding=True,
    temporal_encoding=True,
    temporal_encoding_type="embed",  # linear, embed, fixed-embed
    d_model=32,
    n_heads=1,
    d_ff=64,
    e_layers=1,
    d_layers=1,
    factor=1,
    dropout=0.1,
    activation="relu",
    freq="M",
)

model.fit(y, fh=list(range(1, model.pred_len+1)))

preds = model.predict()
plot_series(y, preds)

In [ ]:
model.get_class_tag("ignores-exogeneous-X")

# MVTSTransformerClassifier

In [ ]:
from sktime.classification.deep_learning import MVTSTransformerClassifier

model = MVTSTransformerClassifier(
    d_model=256,
    n_heads=4,
    num_layers=4,
    dim_feedforward=128,
    dropout=0.1,
    pos_encoding="fixed",
    activation="relu",
    norm="BatchNorm",
    freeze=False,
    num_epochs=10,
	verbose=False,
)

In [ ]:
X_train, y_train = load_unit_test(split="train")
X_test, y_test = load_unit_test(split="test")

model.fit(X_train, y_train)
preds = model.predict(X_test)

preds, y_test

In [ ]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, preds)

# TimesFMForecaster

In [ ]:
y = load_airline()

In [ ]:
from sktime.forecasting.timesfm_forecaster import TimesFMForecaster

forecaster = TimesFMForecaster(
    context_len=64,
    horizon_len=32,
)

In [ ]:
forecaster.fit(y, fh=list(range(1, forecaster.horizon_len)))
y_pred = forecaster.predict()

In [ ]:
plot_series(y, y_pred)

In [ ]:
forecaster.get_class_tag("capability:global_forecasting")

In [ ]:
y_train

In [ ]:
# Preparing the data
y = _make_hierarchical(
    hierarchy_levels=(2, 5), max_timepoints=100, min_timepoints=100, n_columns=1
)
y_train, y_test = train_test_split(
    y, test_size=0.1, shuffle=False
)
y_test = y_test.groupby(level=0).apply(
    lambda x: x.droplevel(0).iloc[:-4]
)

# Loading the model
forecaster = TimesFMForecaster(
    context_len=64,
    horizon_len=32,
)

# fit and forecast
forecaster.fit(y_train, fh=[1, 2, 3, 4])
preds = forecaster.predict(y=y_test)

In [ ]:
preds

# TinyTimeMixerForecaster

In [ ]:
from sktime.forecasting.ttm import TinyTimeMixerForecaster

y = load_airline()

model = TinyTimeMixerForecaster(
    config={
        "context_length": 20,
        "prediction_length": 10,
    },
    training_args={
        "num_train_epochs": 100,
        "output_dir": "test_output",
        "per_device_train_batch_size": 32,
        "report_to": "none",
        "learning_rate": 1e-2,
    },
)

In [ ]:
model.fit(y, fh=list(range(1, 11)))
preds = model.predict()

In [ ]:
plot_series(y, preds)

In [ ]:
forecaster.get_class_tag("capability:global_forecasting")

In [ ]:
# Preparing the data
y = _make_hierarchical(
    hierarchy_levels=(2, 5), max_timepoints=100, min_timepoints=100, n_columns=2
)
y_train, y_test = train_test_split(
    y, test_size=0.1, shuffle=False
)
y_test = y_test.groupby(level=0).apply(
    lambda x: x.droplevel(0).iloc[:-4]
)

# Loading the model
model = TinyTimeMixerForecaster(
    config={
        "context_length": 20,
        "prediction_length": 10,
    },
    training_args={
        "num_train_epochs": 1,
        "output_dir": "test_output",
        "per_device_train_batch_size": 32,
        "report_to": "none",
        "learning_rate": 1e-2,
    },
	broadcasting=False,
)

# fit and forecast
model.fit(y_train, fh=[1, 2, 3, 4])
preds = model.predict(y=y_test)

In [ ]:
preds